Aula 2 - Importação

In [122]:
import pandas as pd
import numpy as np
import plotly.express as px

In [ ]:
df_principal = pd.read_excel("/content/Imersão.xlsx", sheet_name = "Principal")
df_principal.head(20)

In [ ]:
df_total_acoes = pd.read_excel("/content/Imersão.xlsx", sheet_name = "Total_de_acoes")
df_total_acoes

In [11]:
df_principal = df_principal[["Ativo",	"Data",	"Último (R$)",	"Var. Dia (%)"]].copy()

In [12]:
df_principal = df_principal.rename(columns = {"Último (R$)":"valor_final","Var. Dia (%)":"var_dia_pct" }).copy()

In [13]:
df_principal["var_dia"] = df_principal["var_dia_pct"]/100
df_principal["valor_inicial"] = df_principal["valor_final"] / (df_principal["var_dia"] + 1)

In [14]:
df_principal = df_principal.merge(df_total_acoes, left_on="Ativo", right_on="Código", how="left")

In [17]:
df_principal = df_principal.merge(df_total_acoes, left_on="Ativo", right_on="Código", how="left")

In [21]:
df_principal = df_principal.rename(columns = {"Qtde. Teórica_x":"qtd_acoes"}).copy()
df_principal = df_principal.drop(columns=["Código_x", "Código_y", "Qtde. Teórica_y"])

In [22]:
df_principal["var_total_dia"] = (df_principal["valor_final"]-df_principal["valor_inicial"])*df_principal["qtd_acoes"]

In [23]:
pd.options.display.float_format = "{:.2f}".format
df_principal["qtd_acoes"] = df_principal["qtd_acoes"].astype(int)

In [24]:
df_principal["resultado"] = df_principal["var_total_dia"].apply(lambda x: "Subiu" if x>0 else("Desceu" if x<0 else "Estável"))

In [25]:
df_ticker = pd.read_excel("/content/Imersão.xlsx", sheet_name = "Ticker")
df_principal = df_principal.merge(df_ticker, left_on="Ativo", right_on="Ticker", how="left")
df_principal = df_principal.drop(columns = "Ticker")

In [26]:
df_chatgpt = pd.read_excel("/content/Imersão.xlsx", sheet_name = "Chatgpt")
df_principal = df_principal.merge(df_chatgpt, left_on="Nome", right_on="Ticker", how="left")

In [27]:
df_principal = df_principal.drop(columns=["Ativo_y", "Ticker"])

In [ ]:
df_principal = df_principal.rename(columns={"Idade (anos)":"idade"}).copy()
df_principal

In [29]:
df_principal['idade'] = pd.to_numeric(df_principal['idade'], errors='coerce')

In [ ]:
df_principal['idade'] = df_principal['idade'].fillna(0)
df_principal["idade"] = df_principal["idade"].astype(int)
df_principal

In [ ]:
df_principal["categoria"] = df_principal["idade"].apply(lambda x: "Mais de 100" if x>100 else("Menos de 50" if x<50 else "Entre 50 e 100"))
df_principal

In [ ]:
df_principal.replace("[Informação ausente]", None, inplace=True)
df_principal

In [45]:
df_principal["Segmento"] = df_principal["Segmento"].astype(str)

In [ ]:
df_principal

In [53]:
df_principal["idade"].replace(0, np.nan, inplace=True)

In [ ]:
df_principal.loc[83, "Segmento"] = "Energia"
df_principal.loc[84, "Segmento"] = None
df_principal.loc[85, "Segmento"] = None
df_principal.loc[86, "Segmento"] = None

In [ ]:
df_principal["idade"] = df_principal["idade"].astype('Int64')
df_principal

In [ ]:
df_principal.loc[df_principal["idade"].isna(), "categoria"] = None
df_principal

In [ ]:
maior = df_principal["var_total_dia"].max()
print(f"Maior \tR$ {maior:,.2f}")
menor = df_principal["var_total_dia"].min()
print(f"Menor \tR$ {menor:,.2f}")
media = df_principal["var_total_dia"].mean()
print(f"Média \tR$ {media:,.2f}")

media_subiu = df_principal[df_principal["resultado"]=="Subiu"]["var_total_dia"].mean()
print(f"Média de quem subiu \tR$ {media_subiu:,.2f}")

media_desceu = df_principal[df_principal["resultado"]=="Desceu"]["var_total_dia"].mean()
print(f"Média de quem desceu \tR$ {media_desceu:,.2f}")

In [ ]:
df_principal_subiu = df_principal[df_principal["resultado"]=="Subiu"]
df_principal_subiu

In [ ]:
df_analise_segmento = df_principal_subiu.groupby("Segmento")["var_total_dia"].sum().reset_index()
df_analise_segmento

In [ ]:
df_analise_saldo = df_principal.groupby("resultado")["var_total_dia"].sum().reset_index()
df_analise_saldo

In [141]:
fig = px.bar(df_analise_saldo, x="resultado", y="var_total_dia", color="resultado", text="var_total_dia", title="Variação R$ x Resultado",
             labels={
                 "resultado": "Resultados", "var_total_dia": "Variação total por dia"
             })
fig.show()